# Text Augmentation


1. Easy Data Augmentation
    - Random Deletion (직접 구현해보기)
    - Random Insertion (직접 구현해보기)
    - Random Swap
    - Synonym Replacement
2. Back Translation (직접 구현해보기)

In [1]:
import pandas as pd
import random
import pickle
import re

#### 데이터 로드

In [2]:
!gdown https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

Downloading...
From: https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
To: /content/train_original.json
100% 800M/800M [00:13<00:00, 59.9MB/s]


In [3]:
df_news = pd.read_json("./train_original.json")
df_news

,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"
...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]"
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]"
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]"
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]"


In [4]:
train = df_news[['abstractive']].iloc[:100]
train.head()

,abstractive
0,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...
1,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...
2,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...
3,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...
4,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...


In [5]:
train.rename(columns = {'abstractive' : 'contents'}, inplace = True)
train

,contents
0,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...
1,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...
2,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...
3,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...
4,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...
...,...
95,지난 14일 방송된 JTBC '이규연의 하이라이트'에서는 5.18 사건의 진실을 추...
96,(재)백제세계유산센터는 '세계유산 백제역사유적지구와 함께한 추억'이라는 주제로 일러...
97,지난 14일에 KBS1'추적60분'에서 방송된 '환상의 재테크? 태양광 발전의 그늘...
98,체력 소모가 많은 여름에는 '인삼'으로 건강 관리를 하는 사람이 늘고 있어 15일 ...


In [ ]:
sample = train.values[0, 0]
sample

"통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. "

In [ ]:
words = sample.split()
words

['통계청이',
 '발표한',
 "'2018년",
 "사망원인통계'를",
 '보면',
 '지난해',
 '총',
 '사망자',
 '수는',
 '관련',
 '통계를',
 '작성한',
 '이래',
 '가장',
 '많았으며',
 '사망',
 '원인',
 '중',
 '폐렴과',
 '알츠하이머',
 '병의',
 '순위',
 '상승이',
 '두드러졌고',
 '자살에',
 '의한',
 '사망자도',
 '증가했다.']

### Random Deletion

< 사용할 함수 >   
- random.uniform(0, 1)
- random.randint(0, n)

In [ ]:
# 정한 확률만큼 deletion
def random_deletion(words, prob):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > prob:
			new_words.append(word)
	if len(new_words) == 0:
		return [random.choice(words)]
	return new_words

In [ ]:
print(words)

['통계청이', '발표한', "'2018년", "사망원인통계'를", '보면', '지난해', '총', '사망자', '수는', '관련', '통계를', '작성한', '이래', '가장', '많았으며', '사망', '원인', '중', '폐렴과', '알츠하이머', '병의', '순위', '상승이', '두드러졌고', '자살에', '의한', '사망자도', '증가했다.']


In [ ]:
new_words = random_deletion(words, 0.5)
print(new_words)

['통계청이', '지난해', '사망자', '수는', '통계를', '작성한', '이래', '가장', '사망', '원인', '폐렴과', '병의', '순위', '자살에', '의한']


### Random Swap

In [ ]:
def random_swap(words, n):
    new_words = words.copy()
    len_words = len(new_words)
    for _ in range(n):
        idx1, idx2 = random.randint(0, len_words-1), random.randint(0, len_words-1)
        if idx1 == idx2:
            if idx1 <= 1:
                idx2 = random.randint(idx1+1, len_words-1)
            else:
                idx2 = random.randint(0, idx1-1)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]

    return new_words

In [ ]:
new_words = random_swap(words, 2)
print(words)
print(new_words)

['통계청이', '발표한', "'2018년", "사망원인통계'를", '보면', '지난해', '총', '사망자', '수는', '관련', '통계를', '작성한', '이래', '가장', '많았으며', '사망', '원인', '중', '폐렴과', '알츠하이머', '병의', '순위', '상승이', '두드러졌고', '자살에', '의한', '사망자도', '증가했다.']
['많았으며', '발표한', "'2018년", "사망원인통계'를", '보면', '지난해', '총', '사망자', '수는', '관련', '통계를', '작성한', '알츠하이머', '가장', '통계청이', '사망', '원인', '중', '폐렴과', '이래', '병의', '순위', '상승이', '두드러졌고', '자살에', '의한', '사망자도', '증가했다.']


### EDA
- random deletion, random swap 통합

In [ ]:
def EDA(dataframe, alpha_rs=0.1, p_rd=0.1, num_aug=10):
    augmented_sentences = []
    for i in range(len(dataframe)):
        words = dataframe['contents'][i].split()
        num_words = len(words)

        n_swap = random.randint(1, num_aug-1)
        n_del = num_aug - n_swap

        for _ in range(n_swap):
            n_rs = max(1, int(alpha_rs*num_words))
            words_s = random_swap(words, n_rs)
            augmented_sentences.append(" ".join(words_s))

        for _ in range(n_del):
            words_d = random_deletion(words, p_rd)
            augmented_sentences.append(" ".join(words_d))

    train_aug = pd.DataFrame({'contents': augmented_sentences})
    new_train = pd.concat([train, train_aug], ignore_index=True)
    return new_train

def random_deletion(words, prob):
	if len(words) == 1:
		return words[0]

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > prob:
			new_words.append(word)
	if len(new_words) == 0:
		return [random.choice(words)]
	return new_words

def random_swap(words, n):
    new_words = words.copy()
    len_words = len(new_words)
    for _ in range(n):
        idx1, idx2 = random.randint(0, len_words-1), random.randint(0, len_words-1)
        if idx1 == idx2:
            if idx1 <= 1:
                idx2 = random.randint(idx1+1, len_words-1)
            else:
                idx2 = random.randint(0, idx1-1)
        new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    return new_words

In [ ]:
new_train = EDA(train)
new_train

,contents
0,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...
1,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...
2,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...
3,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...
4,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...
...,...
1095,경기도와 경기남부통일교육센터는 21일부터 지작하는 통일전문 '남북상열지사 특별시즌 ...
1096,경기도와 경기남부통일교육센터는 21일부터 지작하는 통일전문 팟캐스트 프로그램인 '남...
1097,경기남부통일교육센터는 21일부터 지작하는 통일전문 팟캐스트 프로그램인 '남북상열지사...
1098,경기도와 경기남부통일교육센터는 통일전문 팟캐스트 프로그램인 '남북상열지사 특별시즌 ...


## Back Translation
1. translator 간의 성능 비교
2. 활용 언어 간의 성능 비교
3. 위 샘플 텍스트 뭉치를 5배로 증강해보기

In [ ]:
!pip install --upgrade translators

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14578 sha256=23341edb12622fe2ddd327f2635499a86636ddba4bc6fd558747bc6f66971d49
  Stored in directory: /root/.cache/pip/wheels/9d/91/30/28e6da53d4f44dc445349b2ffad581968447e4cbc9dd7991b8
Successfully built PyExecJS


In [ ]:
import translators as ts

Using region South Carolina server backend.



In [ ]:
print(ts.translators_pool)

['alibaba', 'apertium', 'argos', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'deepl', 'elia', 'google', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 'judic', 'languageWire', 'lingvanex', 'niutrans', 'mglip', 'mirai', 'modernMt', 'myMemory', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 'translateCom', 'translateMe', 'utibet', 'volcEngine', 'yandex', 'yeekit', 'youdao']


In [ ]:
sample

"통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. "

In [ ]:
sample_eng = ts.translate_text(sample)
sample_eng

In [ ]:
sample_aug = ts.translate_text(sample_eng, to_language='ko')
sample_aug

In [ ]:
def back_translation(dataframe, num_aug=10):
    augmented_sentences = []
    for i in range(len(dataframe)):
        sentence = dataframe['contents'][i]

        for _ in range(num_aug):
            sentence_aug = ts.translate_text(ts.translate_text(sentence), to_language='ko')
            augmented_sentences.append(sentence_aug)

    train_aug = pd.DataFrame({'contents': augmented_sentences})
    new_train = pd.concat([train, train_aug], ignore_index=True)
    return new_train

In [ ]:
new_train = back_translation(train)
new_train